In [ ]:
! code .

### CartPoleでQ学習（Q-learning）を実装・解説【Phythonで強化学習：第1回】
http://neuro-educator.com/rl1/

----------------------------------------------
- ロードマップ：<span class="burk">Q学習</span>　→　DQN　→　A3C → PPO →　HER
- jupyterでgymの結果を表示するのは面倒なので、実行はcodeでする

### コードの中で分からなかった事

#### NumPy でビンの作成、データとビンの対応関係の取得
https://qiita.com/shoz@github/items/86cbce27f0631cfde8c4

- np.random.normal(0, 1, n)
    > 正規分布の行列を作成する。0が中心で1が分散（大きいとばらつく）、nが出力個数

- np.linspace(clip_min, clip_max, num + 1)[1:-1])
    > 線形に等間隔な数列を生成  
clip_maxとclip_minをnumの数で等分割して、行列で返す  
下の例では、1と11との間を10等分させ、はじめの値を抜いて返している  
np.linspace(clip_min, clip_max, num + 1)[0:-1]であれば、1から始まる

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
n = 100
dist = np.random.normal(0, 1, n)
num=10
clip_min=-5
clip_max=5
bins=np.linspace(clip_min, clip_max, num + 1)[1:-1]
plt.hist(dist, bins=bins) 
plt.show()

- np.digitize(dist, bins)
    > データに対応したビンの位置情報が格納された行列を取得  
    distの各要素がbinsの何番目の要素にマッチしているかを得られる

In [ ]:
bin_indice = np.digitize(dist, bins)
bin_indice

#### Qテーブルの作成
最小-1、最大+1の間をランダムに表を作成  
- 行数：num_dizitized**num_state
- 列数：num_action　

In [ ]:
import pandas as pd

num_dizitized=6 #6分割
num_state=4
num_action=2 #env.action_space.n
q_table = np.random.uniform(low=-1, high=1, size=(num_dizitized**num_state,num_action))
df = pd.DataFrame(q_table)

df

## 概要

cartPoleをＱ学習で学習


1. 状態s：倒立振子の状態（State）は下記の4変数
    - カート位置 -2.4～2.4
    - カート速度 -3.0～3.0
    - 棒の角度 　-41.8～41.8
    - 棒の角速度　-2.0～2.0

1. 行動a：実際にとることができる行動（Action）下記2通り→加速度を与える
    - カートを右に押す
    - カートを左に押す

1. 報酬
    - 失敗：棒が20.9度以上傾いたり、カート位置が±2.4以上移動すると失敗
    - 成功：上記以外

1. 目的
    - 状態sに応じて、うまく行動aをとり、棒を立て続ける戦略（policy)を作る


今回の場合200stepの間、立て続ければ成功  
つまり  
a_t=A(s_t)　※時刻tにおいて状態sのときに最適な行動aを返す関数A（policy)  
を求めることがゴールとなります。  

## ライブラリのインポート

In [1]:
# coding:utf-8
import gym  #倒立振子(cartpole)の実行環境
from gym import wrappers  #gymの画像保存
import numpy as np
import time

## Q関数を離散化して定義する関数

In [2]:
# 観測した状態を離散値にデジタル変換する
def bins(clip_min, clip_max, num):
    return np.linspace(clip_min, clip_max, num + 1)[1:-1]
 
# 各値を離散値に変換
def digitize_state(observation):
    cart_pos, cart_v, pole_angle, pole_v = observation
    digitized = [
        np.digitize(cart_pos, bins=bins(-2.4, 2.4, num_dizitized)),
        np.digitize(cart_v, bins=bins(-3.0, 3.0, num_dizitized)),
        np.digitize(pole_angle, bins=bins(-0.5, 0.5, num_dizitized)),
        np.digitize(pole_v, bins=bins(-2.0, 2.0, num_dizitized))
    ]
    return sum([x * (num_dizitized**i) for i, x in enumerate(digitized)])
 

## 行動a(t)を求める関数
次の状態s(t+1)で右に動かすべきか、左に動かすべきか、Q関数の大きい方を選びます。  
ただし、徐々に最適行動のみをとる、ε-greedy法にします。  
※ε-greedy法：基本的には報酬が最大となる行動を選択しますが、ときおりランダムな行動をとります。  
> 広く答えを探さなくなるジレンマに陥るため（探索と利用のジレンマ）  
人間も時々冒険して学ぶ。人によって違うのは、このパラメータ（下記のepsilon）の違い

In [3]:
def get_action(next_state, episode):
           #徐々に最適行動のみをとる、ε-greedy法
    epsilon = 0.5 * (1 / (episode + 1))
    if epsilon <= np.random.uniform(0, 1):
        next_action = np.argmax(q_table[next_state])
    else:
        next_action = np.random.choice([0, 1])
    return next_action

## Qテーブルを更新する関数
Q関数を更新するメソッドを定義  
エージェントは状態をQテーブルと見合わせて、どちらが有効かを判断する  
※DQNではこの表がニューラルネットワークになる

Qテーブル
> 状態（State）の変数は4個で、これを6分割（6^4の1296状態で定義）  
　→　例）カート速度 -3.0～3.0なら[-3.0 , -1.8 , -0.6 , 0.6 , 1.8 , 3.0]で分割  
Q関数は状態（State）×action（2通り）で、[1296×2]の行列（表）で表される

In [4]:
def update_Qtable(q_table, state, action, reward, next_state):
    gamma = 0.99
    alpha = 0.5
    next_Max_Q=max(q_table[next_state][0],q_table[next_state][1] )
    q_table[state, action] = (1 - alpha) * q_table[state, action] +\
            alpha * (reward + gamma * next_Max_Q)
    return q_table

## メイン関数開始 パラメータ設定
はじめに各パラメータを定義します。  
また状態を離散値にして、[1296×2]の行列（表）形式のQ関数を作成します。（-1から1の間で初期値はランダムに作成する）

In [5]:
env = gym.make('CartPole-v0')
max_number_of_steps = 200  #1試行のstep数
num_consecutive_iterations = 100  #学習完了評価に使用する平均試行回数
num_episodes = 2000  #総試行回数
goal_average_reward = 195  #この報酬を超えると学習終了（中心への制御なし）
# 状態を6分割^（4変数）にデジタル変換してQ関数（表）を作成
num_dizitized = 6  #分割数
q_table = np.random.uniform(
    low=-1, high=1, size=(num_dizitized**4, env.action_space.n))
 
total_reward_vec = np.zeros(num_consecutive_iterations)  #ここに各試行の報酬を格納する
final_x = np.zeros((num_episodes, 1))  #ここに学習後、各試行のt=200でのｘの位置を格納する
islearned = 0  #学習が終わったフラグ
isrender = 0  #描画フラグ

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


## メインルーチン
試行数のfor文と、各時間ステップのfor文のネストになっています。  
状態s(t)でa(t)を実行し、観測状態s(t+1)を求めます。  
そのときの棒が立っているかどうかで報酬r(t)を決定します。  
報酬は、195ステップ立たずに終了したら-200の罰則の報酬を与えます。  
こけずに立っていたら、+1の報酬を与えます。  
その後、Q関数を更新し、次の行動a(t+1)を求め、状態s(t)を更新します。  
最後は各ステップごとの情報と、試行終わりの情報を出力し、学習終了条件を満たしているか判定します。  
以上のコードを実行すると、だいたい800試行で学習が収束し、棒がうまく立ちます。  
上記コードを実行した結果をgifで示します。  
40度以上傾くと終了します。  

- observation　：観測データ＝Stateなので、4つの状態array([-0.02029021, -0.03788568, -0.01769319,  0.00148244])
- state　:observationをqテーブルの行番号へ変換
- action　：カートの移動方向

observation, reward, done, inf＝状態,報酬←このリワードは使わない,ゲームオーバかどうか（True or False),インフォメーション

In [6]:
total_reward_vec

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
for episode in range(num_episodes):  #試行数分繰り返す
    # 環境の初期化
    observation = env.reset()
    state = digitize_state(observation)
    action = np.argmax(q_table[state])
    episode_reward = 0
 
    for t in range(max_number_of_steps):  #1試行のループ
        if islearned == 1:  #学習終了したらcartPoleを描画する
#             env.render()
            time.sleep(0.1)
#             print (observation[0])  #カートのx位置を出力
 
        # 行動a_tの実行により、s_{t+1}, r_{t}などを計算する
        observation, reward, done, info = env.step(action)
 
        # 報酬を設定し与える
        if done: #ゲームオーバ　True or Flase　で、　True（ゲームオーバ：こけたの場合）
            if t < 195:#195回連続でこけなかった場合
                reward = -200  #こけたら罰則
            else:
                reward = 1  #立ったまま終了時は罰則はなし
        else:
            reward = 1  #各ステップで立ってたら報酬追加→ゲームオーバーになっていない：こけてない
 
        episode_reward += reward  #報酬を追加
 
        # 離散状態s_{t+1}を求め、Q関数を更新する
        next_state = digitize_state(observation)  #t+1での観測状態を、離散値に変換
        q_table = update_Qtable(q_table, state, action, reward, next_state) #qテーブルを更新
        
        #  次の行動a_{t+1}を求める 
        action = get_action(next_state, episode)    # a_{t+1} 
        
        state = next_state
        
        #終了時の処理
        if done:#ゲームオーバ　True or Flase　で、　True（ゲームオーバ：こけたの場合）
            print( str(episode) + 'Episode finished after' + str(t+1)+'time steps' +' total_reward_mean'+str(total_reward_vec.mean()) ) # total_reward_vec.mean()は全エピソードの報酬の平均
            
            total_reward_vec = np.hstack((total_reward_vec[1:],#np.hstackは配列の連結
                                          episode_reward))  #報酬を記録
#             print(total_reward_vec)
            if islearned == 1:  #学習終わってたら最終のx座標を格納
                final_x[episode, 0] = observation[0]
            break
 
    if (total_reward_vec.mean() >=
            goal_average_reward):  # 直近の100エピソードが規定報酬以上であれば成功
        print('Episode %d train agent successfuly!' % episode)
        islearned = 1
        #np.savetxt('learned_Q_table.csv',q_table, delimiter=",") #Qtableの保存する場合
        if isrender == 0:
            #env = wrappers.Monitor(env, './movie/cartpole-experiment-1') #動画保存する場合
            isrender = 1
    #10エピソードだけでどんな挙動になるのか見たかったら、以下のコメントを外す
    if episode>50:
        if isrender == 0:
#             env = wrappers.Monitor(env, './movie/cartpole-experiment-1') #動画保存する場合
            isrender = 1
        elif islearned==1:
            break

# if islearned:
#     np.savetxt('final_x.csv', final_x, delimiter=",")

0Episode finished after61time steps total_reward_mean-66.23
1Episode finished after80time steps total_reward_mean-67.63
2Episode finished after106time steps total_reward_mean-68.84
3Episode finished after87time steps total_reward_mean-69.79
4Episode finished after13time steps total_reward_mean-70.93
5Episode finished after126time steps total_reward_mean-72.81
6Episode finished after55time steps total_reward_mean-73.56
7Episode finished after90time steps total_reward_mean-75.02
8Episode finished after39time steps total_reward_mean-76.13
9Episode finished after94time steps total_reward_mean-77.75
10Episode finished after87time steps total_reward_mean-78.82
11Episode finished after75time steps total_reward_mean-79.96
12Episode finished after37time steps total_reward_mean-81.22
13Episode finished after120time steps total_reward_mean-82.86
14Episode finished after184time steps total_reward_mean-83.67
15Episode finished after200time steps total_reward_mean-83.84
16Episode finished after200ti